# Q1 - Decision Tree

## a.) Check out dt.py

## b.) Check out dt.py

## c.) See Below for 1C

## d.) 

# !!!!!Refactor THIS!!!!
Every time there is a split, train must look through all *n* values of all *d* features to find
the best split, so we have *nd*. There is at least one split at each level, so total time to
train is O(*ndp*), where *p* is the maximum depth. Predict would be like searching in a red black tree, since it has to get to a leaf node. This takes O(*p*) time as the furthest possible leaf would be at the max depth, and predict would have take make a comparison for every node from and including the root till that leaf.

## Q1 - c.)

In [1]:
#import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Q2 - Model Assessment

## a.) Check out q2.py

## b.) Check out q2.py

## c.) Check out q2.py

## d.) 

### Result:
![Q3 Results](q3.png)
```Python
6   True Test  0.954458  0.828526  0.000000
```

### Analysis

We can see that

# Q3 - Decision Tree Robustness

## a.) 

#### For: 

python3 q3.py 5 10

#### While from my code:
```Python
def get_param(classifier, xFeat, y, xTest, yTest):
    if classifier == "knn":
        clf = GridSearchCV(
            KNeighborsClassifier(),
            {'n_neighbors': range(1, 50, 1)},
            cv=5, scoring='f1_macro')
        clf.fit(xFeat, y['label'])
    else:
        clf = GridSearchCV(
            DecisionTreeClassifier(),
            [{'max_depth': range(1, 20),
              'min_samples_leaf': range(1, 30),
              'criterion': ['entropy', 'gini']}],
            cv=5, scoring='f1_macro')
        clf.fit(xFeat, y)

    optimal_parameter = clf.best_params_
    optimal_parameter_string = str(optimal_parameter)

    print("optimal parameter: " + optimal_parameter_string)

    return optimal_parameter
```

See that the 'n_neighbors' range is (1, 50, 1) for KNN, 

whereas for Decision Tree 'max_depth': range(1, 20), and 'min_samples_leaf': range(1, 30) here

#### Best Knn Parameter:
optimal parameter: {'n_neighbors': 1}

#### Best Decision Tree Parameter:
optimal parameter: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1}

### Analysis

Therefore,

## b.) Check out q3.py

## c.) Check out q3.py

## d.)

### Results:
#### Overall for KNN:

knn auc: 0.72345

knn percent accuracy: 0.85833

#### For 5%:

knn auc: 0.72224

knn percent accuracy: 0.85625

#### For 10%:

knn auc: 0.68906

knn percent accuracy: 0.84375

#### For 20%:

knn auc: 0.70037

knn percent accuracy: 0.85208

#### Overall for Decision Tree:

decision tree auc: 0.88002

decision tree percent accuracy: 0.88125

#### For 5%:

decision tree auc: 0.85679

decision tree percent accuracy: 0.86458

#### For 10%:

decision tree auc: 0.86918

decision tree percent accuracy: 0.9

#### For 20%:

decision tree auc: 0.86536

decision tree percent accuracy: 0.86667

### Analysis

Therefore,